In [0]:
from pyspark.sql.functions import monotonically_increasing_id, col, struct, expr, array
from itertools import product

# Programming Assignment 2

Wyatt Blair

12/2/2024

Using the environmental data for each of the provinces in Canada, and weighting each piece of data by the number of cities in the province, calculate the mean temperature and mean precipitation for all of Canada for annual and each month.

## Load the data

In [0]:
%sql
use catalog `hive_metastore`; select * from `default`.`class_9___12____data_for_programming___environmental___vshort_3_csv` limit 100;

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,_c15,_c16
Alberta,ANNUAL,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,YEARS,# CITIES,null
Average Temperature (F),36.8,10.6,15.8,25.3,39.1,49.5,56.7,60.9,59.2,50,39.2,23.3,13.8,24,245,null
Average High Temperature (F),48.3,21.2,27,36.2,51.2,62.1,68.8,73.6,72.3,62.5,50.6,32.6,23.8,25,236,null
Average Low Temperature (F),25.8,0.9,5,14.5,27.4,36.9,44.7,48.5,46.4,37.7,28.2,14.1,4.4,25,236,null
Average Precipitation (in),18.2,0.9,0.7,0.9,1.1,2,3.2,3,2.3,1.7,0.9,0.9,0.8,24,277,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
British Columbia,ANNUAL,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,YEARS,# CITIES,null
Average Temperature (F),43.7,27.2,30.5,36.7,43.8,50.9,56.8,61.2,60.8,54,44.3,34,27.5,24,471,null
Average High Temperature (F),52.2,32.9,37.6,45.1,53.5,61.3,67.1,72.2,72,64.3,52,39.4,32.8,24,469,null
Average Low Temperature (F),35.2,21.5,23.4,28.2,34.1,40.6,46.5,50.1,49.5,43.7,36.7,28.5,22.3,24,469,null


In [0]:
raw_data = _sqldf.withColumn('row_index', monotonically_increasing_id())

## Clean up data

In [0]:
provinces = raw_data.filter(col('row_index') % 6 == 0).drop('row_index').select('_c0').withColumnRenamed('_c0', 'Province')
n_provinces = provinces.count()

display(provinces)

Province
Alberta
British Columbia
Manitoba
New Brunswick
Newfoundland
Northwest Territories
Nova Scotia
Nunavut
Ontario
Prince Edward Island


In [0]:
headers = raw_data.collect()[0][1:-2]
headers = [header.strip().replace('# ', 'N_') for header in headers]
print(headers)

['ANNUAL', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'YEARS', 'N_CITIES']


In [0]:
measurements = raw_data.filter(
  (col('row_index') > 0)
  &
  (col('row_index') < 5)
  ).select('_c0').withColumnRenamed('_c0', 'Measurement')
display(measurements)

Measurement
Average Temperature (F)
Average High Temperature (F)
Average Low Temperature (F)
Average Precipitation (in)


In [0]:
data = raw_data.filter(
  (col('row_index') % 6 > 0)
  &
  (col('row_index') % 6 < 5)
  )
data = data.drop('_c0', '_c16', 'row_index')

for i, header in enumerate(headers):
  data = data.withColumnRenamed(f'_c{i+1}', header.strip())

display(data)

ANNUAL,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,YEARS,N_CITIES
36.8,10.6,15.8,25.3,39.1,49.5,56.7,60.9,59.2,50,39.2,23.3,13.8,24,245
48.3,21.2,27,36.2,51.2,62.1,68.8,73.6,72.3,62.5,50.6,32.6,23.8,25,236
25.8,0.9,5,14.5,27.4,36.9,44.7,48.5,46.4,37.7,28.2,14.1,4.4,25,236
18.2,0.9,0.7,0.9,1.1,2,3.2,3,2.3,1.7,0.9,0.9,0.8,24,277
43.7,27.2,30.5,36.7,43.8,50.9,56.8,61.2,60.8,54,44.3,34,27.5,24,471
52.2,32.9,37.6,45.1,53.5,61.3,67.1,72.2,72,64.3,52,39.4,32.8,24,469
35.2,21.5,23.4,28.2,34.1,40.6,46.5,50.1,49.5,43.7,36.7,28.5,22.3,24,469
49,7.1,4.3,4,3.3,2.8,2.8,2.2,2.2,2.9,5.3,6.9,6.2,25,517
34.6,-0.3,5.9,18.5,36.2,49.7,59.6,64.7,62.9,52.1,39.1,20.7,5.6,25,144
44.6,9.2,15.9,28.5,47.1,61.6,70.7,75.8,74.4,62.6,48.1,28.3,14.1,23,140


In [0]:
multi_index = provinces.crossJoin(measurements)
display(multi_index)

Province,Measurement
Alberta,Average Temperature (F)
Alberta,Average High Temperature (F)
Alberta,Average Low Temperature (F)
Alberta,Average Precipitation (in)
British Columbia,Average Temperature (F)
British Columbia,Average High Temperature (F)
British Columbia,Average Low Temperature (F)
British Columbia,Average Precipitation (in)
Manitoba,Average Temperature (F)
Manitoba,Average High Temperature (F)


## Create DataFrame with multi-index for organization's sake

In [0]:
multi_index = multi_index.withColumn("row_index", monotonically_increasing_id())
full_data = data.withColumn("row_index", monotonically_increasing_id())

full_data = multi_index.join(full_data, on="row_index").drop("row_index")

for header in headers:
  full_data = full_data.withColumn(header, full_data[header].cast('double'))

display(full_data)

Province,Measurement,ANNUAL,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,YEARS,N_CITIES
Alberta,Average Temperature (F),36.8,10.6,15.8,25.3,39.1,49.5,56.7,60.9,59.2,50.0,39.2,23.3,13.8,24.0,245.0
Alberta,Average High Temperature (F),48.3,21.2,27.0,36.2,51.2,62.1,68.8,73.6,72.3,62.5,50.6,32.6,23.8,25.0,236.0
Alberta,Average Low Temperature (F),25.8,0.9,5.0,14.5,27.4,36.9,44.7,48.5,46.4,37.7,28.2,14.1,4.4,25.0,236.0
Alberta,Average Precipitation (in),18.2,0.9,0.7,0.9,1.1,2.0,3.2,3.0,2.3,1.7,0.9,0.9,0.8,24.0,277.0
British Columbia,Average Temperature (F),43.7,27.2,30.5,36.7,43.8,50.9,56.8,61.2,60.8,54.0,44.3,34.0,27.5,24.0,471.0
British Columbia,Average High Temperature (F),52.2,32.9,37.6,45.1,53.5,61.3,67.1,72.2,72.0,64.3,52.0,39.4,32.8,24.0,469.0
British Columbia,Average Low Temperature (F),35.2,21.5,23.4,28.2,34.1,40.6,46.5,50.1,49.5,43.7,36.7,28.5,22.3,24.0,469.0
British Columbia,Average Precipitation (in),49.0,7.1,4.3,4.0,3.3,2.8,2.8,2.2,2.2,2.9,5.3,6.9,6.2,25.0,517.0
Manitoba,Average Temperature (F),34.6,-0.3,5.9,18.5,36.2,49.7,59.6,64.7,62.9,52.1,39.1,20.7,5.6,25.0,144.0
Manitoba,Average High Temperature (F),44.6,9.2,15.9,28.5,47.1,61.6,70.7,75.8,74.4,62.6,48.1,28.3,14.1,23.0,140.0


## Weight values by `N_CITIES` column

In [0]:
non_numerical_cols = ["Province", "Measurement", "YEARS", "N_CITIES"]
numerical_cols = [c for c in full_data.columns if c not in non_numerical_cols]

meas_df = full_data
for numerical_col in numerical_cols:
  meas_df = meas_df.withColumn(f'WEIGHTED_{numerical_col}', col(numerical_col) * col('N_CITIES'))
  meas_df = meas_df.drop(numerical_col)

meas_df = meas_df.drop('Province', 'YEARS')
display(meas_df)

Measurement,N_CITIES,WEIGHTED_ANNUAL,WEIGHTED_JAN,WEIGHTED_FEB,WEIGHTED_MAR,WEIGHTED_APR,WEIGHTED_MAY,WEIGHTED_JUN,WEIGHTED_JUL,WEIGHTED_AUG,WEIGHTED_SEP,WEIGHTED_OCT,WEIGHTED_NOV,WEIGHTED_DEC
Average Temperature (F),245.0,9016.0,2597.0,3871.0,6198.5,9579.5,12127.5,13891.5,14920.5,14504.0,12250.0,9604.0,5708.5,3381.0
Average Temperature (F),236.0,11398.8,5003.2,6372.0,8543.2,12083.2,14655.6,16236.8,17369.6,17062.8,14750.0,11941.6,7693.6,5616.8
Average Temperature (F),236.0,6088.8,212.4,1180.0,3422.0,6466.4,8708.4,10549.2,11446.0,10950.4,8897.2,6655.2,3327.6,1038.4
Average Temperature (F),277.0,5041.4,249.3,193.89999999999998,249.3,304.70000000000005,554.0,886.4000000000001,831.0,637.0999999999999,470.9,249.3,249.3,221.60000000000002
Average Temperature (F),471.0,20582.7,12811.199999999999,14365.5,17285.7,20629.8,23973.899999999998,26752.8,28825.2,28636.8,25434.0,20865.3,16014.0,12952.5
Average Temperature (F),469.0,24481.800000000003,15430.099999999999,17634.4,21151.9,25091.5,28749.699999999997,31469.899999999998,33861.8,33768.0,30156.699999999997,24388.0,18478.6,15383.199999999999
Average Temperature (F),469.0,16508.800000000003,10083.5,10974.599999999999,13225.8,15992.900000000001,19041.4,21808.5,23496.9,23215.5,20495.300000000003,17212.300000000003,13366.5,10458.7
Average Temperature (F),517.0,25333.0,3670.7,2223.1,2068.0,1706.1,1447.6,1447.6,1137.4,1137.4,1499.3,2740.1,3567.3,3205.4
Average Temperature (F),144.0,4982.400000000001,-43.199999999999996,849.6,2664.0,5212.8,7156.8,8582.4,9316.800000000001,9057.6,7502.400000000001,5630.400000000001,2980.7999999999997,806.4
Average Temperature (F),140.0,6244.0,1288.0,2226.0,3990.0,6594.0,8624.0,9898.0,10612.0,10416.0,8764.0,6734.0,3962.0,1974.0


## Group by `Measurement` column and take the weighted average

In [0]:
result_df = meas_df.groupBy("Measurement").sum()
for field in result_df.schema.fields:

  if field.name not in ['Measurement', 'sum(N_CITIES)']:
    result_df = result_df.withColumn(f'avg({field.name.removeprefix("sum(WEIGHTED_").removesuffix(")")})', col(field.name) / col('sum(N_CITIES)'))
    result_df = result_df.drop(field.name)

result_df = result_df.drop('sum(N_CITIES)')
result_df = result_df.filter((col('Measurement') == 'Average Temperature (F)') | (col('Measurement') == 'Average Precipitation (in)'))
display(result_df)


Measurement,avg(ANNUAL),avg(JAN),avg(FEB),avg(MAR),avg(APR),avg(MAY),avg(JUN),avg(JUL),avg(AUG),avg(SEP),avg(OCT),avg(NOV),avg(DEC)
Average Temperature (F),38.84745011086475,14.209839246119733,16.85873059866962,22.82649667405765,30.637305986696234,37.38262195121951,42.63414634146341,45.797366962305986,44.99539911308204,39.07818736141907,31.650083148558757,22.222394678492243,15.679046563192905
Average Precipitation (in),34.69959394610557,4.680324843115541,7.785529715762274,16.088261351052047,27.43831672203765,36.7531561461794,43.69579180509413,47.362643041712815,45.83252122554448,38.91328903654485,30.0608711701735,19.47626430417128,9.435289774824662
